# Topic Analysis Using LDA (Combination of Gensim and Sklearn)

## Add module to system path

In [1]:
import sys
sys.path.append("../Modules/")

## Import libraries

In [2]:
import pandas as pd
import LDAGensimTopicAnlalysis as lda
import warnings
import time
import pickle
import gensim
import pyLDAvis.gensim

C:\Users\User\Anaconda3\lib\site-packages\thinc\neural\train.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from .optimizers import Adam, linear_decay
C:\Users\User\Anaconda3\lib\site-packages\thinc\check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, Sized, Iterable, Callable
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perc

## Import data

In [3]:
df_all = pd.read_csv('../Data/All-seasons.csv')
df_all.sample(10)

,Season,Episode,Character,Line
46818,4,8,Kyle,"Yeah, you can have a drink of water. The dispe..."
58093,6,13,Sixth Grader 1,Lord of the Rings? Ha! That movie's gay!\n
53863,5,12,Mr. Garrison,That's great! And now we can sell all their ho...
33765,2,6,Jimbo,"Oh. Yeah, we sure were.\n"
49665,4,17,Kyle,"It's only three days until Christmas, you guys..."
25049,17,9,Niles,That's right Tom. Mall officials have said the...
35824,2,12,Stan,Huh-I dunno.\n
22016,16,10,Randy,You said we wouldn't be prepared to get rid of...
68589,9,5,Umpire,Strike 1!\n
14208,14,4,Kyle,What? How the hell do I only have 37 friends?\n


---

## Cleaning text

In [4]:
# Test to see how the prepare text function actually transforms the text
for text in df_all[df_all['Line'].isnull()==False]['Line'].sample(5):
    print('\n')
    print('Original:', text)
    print('Cleaned:', lda.prepare_text_for_lda(text))



Original: Okay, see, I am the mayor of Smiley Town, and so I will ask the questions around hyah!

Cleaned: okay see the mayor smiley town and will ask the question around hyah


Original: Moving?

Cleaned: move


Original: She's a candy cane to wiggle

Cleaned: she candy cane wiggle


Original: Awesome.

Cleaned: awesome


Original: Oooh.

Cleaned: oooh


In [12]:
df_text = pd.DataFrame(df_all[(df_all['Season']=='16') & (df_all['Episode']=='1')]['Line']).dropna()
df_text.drop_duplicates(inplace=True)
df_text.sample(5)

,Line
20051,"Aw, he's dead?\n"
19984,Why?\n
20045,"Uh-uh, go on.\n"
20012,"Clyde! Clyde! Clyde, I want you to know I don..."
20033,"It's the Toilet Safety Administration, hun. Th..."


In [13]:
start_time = time.time()
df_text['clean_text'] = df_text['Line'].apply(lambda x: lda.prepare_text_for_lda(x))
end_time = time.time()
print('Time to clean text:', (end_time - start_time)/60)

df_text.sample(5)

Time to clean text: 1.782490646839142


,Line,clean_text
19998,Nuh-uh.\n,nuhuh
20077,This is inhumane.\n,this inhumane
20122,"Anyways, he says I'm getting nothing anyway, s...",anyways say get nothing anyway then you can
20018,"Yeah, yes. That's right.\n",yeah yes that right
20067,The address here is 260 Avenue de los Mexicano...,the address here 260 avenue los mexicano


---

## Vectorize text

In [14]:
lda.vect_train_text(df_text, 'clean_text', ngram_range=(3,3), save_path='../Resources/lda')

C:\Users\User\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


---

## Train LDA Model

In [15]:
corpus = pickle.load(open('../Resources/lda_corpus.pkl', 'rb'))
dictionary = gensim.corpora.Dictionary.load('../Resources/lda_dictionary.gensim')
vect = pickle.load(open('../Resources/lda_vect.pkl', 'rb'))

In [16]:
lda.train_lda_model(n_topics=5, corpus=corpus, dictionary=dictionary, save_path='../Resources/lda5')

C:\Users\User\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\User\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Done fitting LDA with 5 topics... Time elapsed 0.03150457143783569


---

## Prepare model data for visualisation

In [17]:
start_time = time.time()
lda5_model = gensim.models.ldamodel.LdaModel.load('../Resources/lda5.gensim')
lda5_display = pyLDAvis.gensim.prepare(lda5_model, corpus, dictionary, sort_topics=False)
end_time = time.time()
print('Time to prepare model data:', (end_time-start_time)/60)

C:\Users\User\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\User\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\User\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior an

Time to prepare model data: 0.5829641977945964


## Display results

In [18]:
pyLDAvis.display(lda5_display)

---